# Agent란?

**Agent(에이전트)** 는 주어진 목표를 달성하기 위해 환경과 상호작용하며 의사 결정을 내리고 행동을 취하는 지능형 AI입니다.  
LangChain에서 Agent는 **LLM의 추론 능력** 과 **외부 도구들** 을 연결하여 복잡한 작업을 수행할 수 있게 합니다.

### Agent의 핵심 구성요소
1. **LLM (Large Language Model)**
   - Agent의 두뇌 역할
   - 자연어 이해와 추론
   - 도구 선택 결정

2. **Tools (도구)**
   - 특정 작업을 수행하는 함수
   - 계산기, 검색엔진, API 호출 등
   - Agent가 실제 행동을 취하는 수단

3. **Agent Executor (실행기)**
   - LLM과 도구 간의 상호작용 관리
   - 실행 흐름 제어
   - 에러 처리 및 재시도 로직

## 도구(Tool) 정의와 바인딩

### 도구의 3가지 핵심 구성요소

모든 LangChain 도구는 다음 세 가지 요소로 구성됩니다:

1. **name (이름)**: 도구를 식별하는 고유한 이름
2. **description (설명)**: 도구의 기능과 사용 시점을 설명
3. **args_schema (인자 스키마)**: 입력 파라미터의 형식 정의


In [1]:
# 환경 설정
import os
from dotenv import load_dotenv

# .env 파일에서 환경변수 로드
load_dotenv()

# API 키 확인
openai_api_key = os.getenv('OPENAI_API_KEY')
print(f"OpenAI API 키가 설정되어 있습니다: {openai_api_key[:10]}...")


OpenAI API 키가 설정되어 있습니다: sk-proj-WS...


실습을 위한 사용자 정의 도구 만들기

`@tool` 데코레이터를 사용하여 간단하게 사용자 정의 도구를 만들 수 있습니다

In [2]:
from langchain.tools import tool

# @tool 데코레이터를 사용한 도구 정의
@tool
def add_numbers(a: int, b: int) -> int:
    """두 숫자를 더합니다. 수학적 덧셈이 필요할 때 사용하세요."""
    return a + b

@tool
def multiply_numbers(a: int, b: int) -> int:
    """두 숫자를 곱합니다. 수학적 곱셈이 필요할 때 사용하세요."""
    return a * b

@tool
def divide_numbers(a: float, b: float) -> float:
    """두 숫자를 나눕니다. 수학적 나눗셈이 필요할 때 사용하세요. 0으로 나누는 것은 불가능합니다."""
    if b == 0:
        raise ValueError("0으로 나눌 수 없습니다")
    return a / b

# 도구 리스트 생성
math_tools = [add_numbers, multiply_numbers, divide_numbers]

# 도구 정보 확인
for tool in math_tools:
    print(f"도구명: {tool.name}")
    print(f"설명: {tool.description}")
    print(f"구조: {tool.args_schema.schema()}")
    print("-" * 50)


도구명: add_numbers
설명: 두 숫자를 더합니다. 수학적 덧셈이 필요할 때 사용하세요.
구조: {'description': '두 숫자를 더합니다. 수학적 덧셈이 필요할 때 사용하세요.', 'properties': {'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}, 'required': ['a', 'b'], 'title': 'add_numbers', 'type': 'object'}
--------------------------------------------------
도구명: multiply_numbers
설명: 두 숫자를 곱합니다. 수학적 곱셈이 필요할 때 사용하세요.
구조: {'description': '두 숫자를 곱합니다. 수학적 곱셈이 필요할 때 사용하세요.', 'properties': {'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}, 'required': ['a', 'b'], 'title': 'multiply_numbers', 'type': 'object'}
--------------------------------------------------
도구명: divide_numbers
설명: 두 숫자를 나눕니다. 수학적 나눗셈이 필요할 때 사용하세요. 0으로 나누는 것은 불가능합니다.
구조: {'description': '두 숫자를 나눕니다. 수학적 나눗셈이 필요할 때 사용하세요. 0으로 나누는 것은 불가능합니다.', 'properties': {'a': {'title': 'A', 'type': 'number'}, 'b': {'title': 'B', 'type': 'number'}}, 'required': ['a', 'b'], 'title': 'divide_numbers', 'type': 'object'}
----------------------------

C:\Users\jtw57\AppData\Local\Temp\ipykernel_13900\442671679.py:28: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  print(f"구조: {tool.args_schema.schema()}")


LLM에 도구 바인딩

`bind_tools()` 메서드를 사용하여 LLM에 도구를 연결합니다:


In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chat_models import init_chat_model
# Agent 프롬프트 생성
# system 프롬프트 : LLM 이 동작될 방향 설정 
# user 프롬프트 : input 값 
# Ai assistant : input 값에 대해 LLM 이 응답한 내용
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful assistant",
        ),
        ("user", "{input}"),
        
        # LLM 이 전달하는 명령서가 들어갈 위치
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

# 모델 생성
llm = init_chat_model("gpt-4o-mini", model_provider="openai")

계산기 Agent 구현

## AgentExecutor
- 에이전트를 실행하는 역할, 에이전트가 어떤 작업을 수행할지 결정하고, 필요한 도구를 사용하며,  
최종답변을 만들어내는 전체과정을 관리하는 실행기

**핵심역할**
- 에이전트와 도구 연결 :   
생성된 Agent와 그 Agent가 사용할 수 있는 도구 목록을 받아 이 둘을 연결한다.

- 실행 루프 관리 :   
에이전트가 어떤 도구를 사용할지 결정하고, 실행한 뒤 결과를 에이전트에게 전달하는 과정을 반복한다. (최종답변이 나올때까지)

- 중간과정 기록 :   
에이전트가 중간에 어떤 생각을 하고 어떤 행동을 취했으며 결과(Observation)이 어땠는지 기록하고 관리한다.

#### 주요매개변수
**핵심 요소 (필수)**  
- `agent`: 실행의 주체인 "뇌"입니다. 어떤 판단을 내릴지 결정합니다.  
- `tools`: 에이전트가 사용할 수 있는 "손발"과 같은 도구들의 목록입니다.

**부가 기능 (선택)**    
- `verbose`: 에이전트의 생각(Thought), 행동(Action) 등 모든 실행 과정을 화면에 상세히 출력해 디버깅을 쉽게 도와줍니다.  
- `return_intermediate_steps`: 실행 과정을 화면에 출력하는 대신, 그 기록을 결과값으로 직접 돌려받고 싶을 때 사용합니다. 
- `memory`: 이전 대화 내용을 기억하게 만들어, 문맥을 이해하는 연속적인 대화가 가능하도록 합니다. 
- `max_iterations`: 에이전트가 무한 루프에 빠지지 않도록 최대 반복 횟수를 제한합니다.  
- `max_execution_time`: 전체 실행 시간이 너무 길어지지 않도록 시간을 제한합니다.
- `handle_parsing_errors`: 에이전트가 내린 결정(예: 도구 이름)을 해석하다 오류가 났을 때 어떻게 대처할지 정합니다.

In [4]:
from langchain.agents import AgentExecutor, create_tool_calling_agent

# 1. Agent 생성
# tool_calling_agent : 구조화된 tool call(Json)을 모델이 출력 -> AgentExecutor한테 전달
math_agent = create_tool_calling_agent(llm, math_tools, prompt)

# 2. Agent Executor 생성
math_executor = AgentExecutor(
    agent=math_agent,
    tools=math_tools,
    verbose=True,  # 실행 과정을 자세히 보여줍니다
    handle_parsing_errors=True  # 파싱 에러를 자동으로 처리합니다
)

print("계산기 Agent가 생성되었습니다!")

계산기 Agent가 생성되었습니다!


### 계산기 Agent 테스트

간단한 계산부터 복잡한 계산까지 테스트해보겠습니다:

In [5]:
# 간단한 계산 테스트
result = math_executor.invoke({
    "input": "25*4 에 대해 계산하세요"
})

print("\n=== 최종 결과 ===")
print(result["output"])




> Entering new AgentExecutor chain...

Invoking: `multiply_numbers` with `{'a': 25, 'b': 4}`


10025 곱하기 4는 100입니다.

> Finished chain.

=== 최종 결과 ===
25 곱하기 4는 100입니다.


In [7]:
# 복잡한 계산 테스트
result = math_executor.invoke({
    "input": "((15 + 25)*3)/6 을 계산하세요"
})

print("\n=== 최종 결과 ===")
print(result["output"])




> Entering new AgentExecutor chain...

Invoking: `add_numbers` with `{'a': 15, 'b': 25}`


40
Invoking: `multiply_numbers` with `{'a': 30, 'b': 3}`


90
Invoking: `divide_numbers` with `{'a': 90, 'b': 6}`


15.0계산 결과는 다음과 같습니다:

1. \(15 + 25 = 40\)
2. \((40 \times 3) = 120\)
3. \(\frac{120}{6} = 20.0\)

따라서, \(((15 + 25) \times 3) / 6 = 20\)입니다.

> Finished chain.

=== 최종 결과 ===
계산 결과는 다음과 같습니다:

1. \(15 + 25 = 40\)
2. \((40 \times 3) = 120\)
3. \(\frac{120}{6} = 20.0\)

따라서, \(((15 + 25) \times 3) / 6 = 20\)입니다.


## 검색 Agent 구현

이제 실시간 정보 검색이 가능한 Agent를 만들어보겠습니다.

### 검색 도구 설정

**주의**: Tavily 검색 도구를 사용하려면 API 키가 필요합니다. 
- 발급 주소: https://app.tavily.com/
- `.env` 파일에 `TAVILY_API_KEY=your_key_here` 추가


In [8]:
# Tavily 검색 도구 설정 (API 키가 필요합니다)
from langchain_community.tools.tavily_search import TavilySearchResults

# 검색 도구 생성
search_tool = TavilySearchResults(
    max_results=3,
    include_answer=True,
    include_raw_content=False,
    search_depth="basic"
)

# 검색 도구 리스트
search_tools = [search_tool]

print("검색 도구가 준비되었습니다.")


검색 도구가 준비되었습니다.


C:\Users\jtw57\AppData\Local\Temp\ipykernel_13900\1950780117.py:5: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-tavily package and should be used instead. To use it run `pip install -U :class:`~langchain-tavily` and import as `from :class:`~langchain_tavily import TavilySearch``.
  search_tool = TavilySearchResults(


In [9]:
# 검색 Agent 생성
search_agent = create_tool_calling_agent(llm, search_tools, prompt)
search_executor = AgentExecutor(
    agent=search_agent,
    tools=search_tools,
    verbose=True,
    handle_parsing_errors=True
)

print("검색 Agent가 생성되었습니다!")


검색 Agent가 생성되었습니다!


In [10]:
# 검색 Agent 테스트
result = search_executor.invoke({
    "input": "2025년 AI 트렌드에 대해 알려주세요."
})

print("=== 최종 결과 ===")
print(result["output"])




> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': '2025 AI trends'}`


[{'title': '5 AI Trends Shaping Innovation and ROI in 2025 | Morgan Stanley', 'url': 'https://www.morganstanley.com/insights/articles/ai-trends-reasoning-frontier-models-2025-tmt', 'content': '[Skip to content](https://www.morganstanley.com/insights/articles/ai-trends-reasoning-frontier-models-2025-tmt#maincontent) [](https://www.morganstanley.com/insights/articles/ai-trends-reasoning-frontier-models-2025-tmt#mobileNav) *   [Sustainable Investing](https://www.morganstanley.com/about-us/sustainability-at-morgan-stanley) *   [Email this](mailto:?body=Did%20you%20know,%20%22Recent%20AI%20advancements%20will%20harness%20the%20power%20of%20Jevons%20Paradox,%20to%20drive%20the%20long-term%20demand%20for%20AI%20and%20further%20increase%20the%20total%20addressable%20market%20for%20all%20participants%20in%20the%20ecosystem.%22.%20You%20can%20read%20more%20about%20it%20here:%20htt

## 종합 Agent 구현

계산과 검색 기능을 모두 가진 종합 Agent를 만들어보겠습니다.

In [11]:
# 모든 도구를 결합
all_tools = math_tools + [search_tool]

print(f"총 {len(all_tools)}개의 도구가 준비되었습니다:")

총 4개의 도구가 준비되었습니다:


종합 Agent 생성

In [12]:
comprehensive_agent = create_tool_calling_agent(llm, all_tools, prompt)
comprehensive_executor = AgentExecutor(
    agent=comprehensive_agent,
    tools=all_tools,
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=10  # 최대 반복 횟수 제한
)

복합 작업 테스트

In [13]:
result = comprehensive_executor.invoke({
    "input": "916에서 15를 빼고, 그날이 생일인 한국 연예인을 찾아주세요"
})

print("=== 최종 결과 ===")
print(result["output"])



> Entering new AgentExecutor chain...

Invoking: `add_numbers` with `{'a': 916, 'b': -15}`


901
Invoking: `tavily_search_results_json` with `{'query': 'Korean celebrities birthdays on 901'}`


[{'title': "Famous People's Birthdays, January, South Korea Celebrity Birthdays", 'url': 'https://www.bornglorious.com/south_korea/birthday/?pd=01', 'content': '1 Jennie (singer) · 2 Lee Seung-gi · 3 Jisoo (singer, born 1995) · 4 Seung-Hui Cho · 5 Moon Jae-in · 6 Kai (entertainer, born 1994) · 7 Son Ye-jin · 8 Song Kang-ho.', 'score': 0.58408195}, {'title': "Famous People's Birthdays, September 01, South Korea Celebrity ...", 'url': 'https://www.bornglorious.com/south_korea/birthday/?pd=0901', 'content': '1 Jungkook · 2 Roh Moo-hyun · 3 Amy Anderson (comedian) · 4 Hwang Jung-min · 5 Woo Hye-rim · 6 Kim Jae-hwa · 7 Mikyung Sung · 8 Lee Jung-hyun (politician).', 'score': 0.5803764}, {'title': 'South Korea | Famous Birthdays', 'url': 'https://www.famousbirthdays.com/birthplace/southkorea.html', '